In [83]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)


In [34]:
from langchain_community.document_loaders import PyPDFLoader

file_path = (
    "../data/atomic-habits.pdf"
)
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()

content = []
for i in range(len(pages)):
    content.append(pages[i].page_content)



In [ ]:

cleaned_list = []
cleaning_prompt = '''Here's a prompt you can use with the OpenAI API to clean your text efficiently:
You are a text cleaning and formatting assistant. Your task is to clean and format the following text from a book summary, maintaining its original meaning and structure while improving readability. Please follow these guidelines: 
Remove any special characters or symbols that aren't standard punctuation.
Fix any spacing issues, ensuring single spaces between words and appropriate line breaks.
Correct obvious spelling errors and merge split words (e.g., 'habit s' to 'habits').
Format bullet points consistently using '-' as the bullet character.
Preserve the original structure of sections and subsections.
Capitalize the first letter of each sentence and proper nouns.
Ensure consistent formatting for titles and subtitles.
Retain the core content and meaning of the text.
Remove any emojis and non text stuff including asteriks.
Remove any new line characters like '\n' and words such as here'/s and it'/s, fix them and make here's and it's.
Also don't say "Here's the text to clean and format" in the response. Just give back the cleaned and formatted text. Don't include the prompt in the response.

Please provide the cleaned and formatted version of this text.
Here's the text to clean and format:
'''

for item in content:
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": cleaning_prompt},
                {"role": "user", "content": f"Clean and format the text:\n\n{item}"}
            ]
        )
        cleaned_text = completion.choices[0].message.content
        cleaned_list.append(cleaned_text)

# Print the response
print(completion.choices[0].message.content)

In [ ]:
cleaned_list

In [37]:
import re

def remove_new_lines_and_slashes(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\u2014', ' ', text)
    text = re.sub(r"\\'", "'", text)
    return text

cleaned_list = [remove_new_lines_and_slashes(item) for item in cleaned_list]

In [ ]:
cleaned_list

In [ ]:
import json

# This will create a json file and write atomic habits text to it
with open('atomic_habits_notes.json', 'w') as f:
    json.dump({"notes": cleaned_list}, f, indent=2)

print("Notes saved to atomic_habits_notes.json")

In [ ]:
import json
from openai import OpenAI

def structure_atomic_habits_with_ai(input_file, output_file, api_key):
    # Initialize OpenAI client
    client = OpenAI(api_key=api_key)

    # Read the original JSON file
    with open(input_file, 'r') as file:
        data = json.load(file)

    structuring_prompt = """
    You are an AI assistant tasked with structuring content from the book "Atomic Habits" into a specific JSON format. The content is currently unstructured. Your task is to organize it into the following structure:

    {
      "title": "Atomic Habits: Tiny Changes, Remarkable Results",
      "laws": [
        {
          "name": "The 1st Law",
          "description": "Make It Obvious",
          "chapters": [
            {
              "number": <chapter number>,
              "title": "<chapter title>",
              "content": "<chapter content>"
            },
            // ... other chapters for this law
          ]
        },
        // ... other laws (2nd, 3rd, 4th)
      ],
      "advanced_tactics": {
        "title": "Advanced Tactics: How to Go from Being Merely Good to Being Truly Great",
        "chapters": [
          {
            "number": <chapter number>,
            "title": "<chapter title>",
            "content": "<chapter content>"
          },
          // ... other advanced tactics chapters
        ]
      },
      "conclusion": {
        "title": "Conclusion: The Secret to Results That Last",
        "content": "<conclusion content>"
      }
    }

    Please structure the following content into this JSON format, cleaning and formatting the text as needed. Ensure that each chapter is placed under the correct law or section, and that the conclusion is properly placed.

    Here's the content to structure:
    """

    # Combine all notes into a single string
    all_content = " ".join(data['notes'])

    # Use OpenAI to structure the content
    completion = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": structuring_prompt},
            {"role": "user", "content": all_content}
        ]
    )

    structured_content = completion.choices[0].message.content

    # Write the structured content to a new JSON file
    with open(output_file, 'w') as file:
        file.write(structured_content)

    print(f"Structured JSON has been written to {output_file}")


structure_atomic_habits_with_ai('atomic_habits_notes.json', 'ai_structured_atomic_habits.json', api_key)

In [ ]:
import json
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Function to load and preprocess the JSON data
def load_and_preprocess_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    documents = []
    
    def extract_content(item, parent_key=''):
        nonlocal documents
        if isinstance(item, dict):
            for key, value in item.items():
                new_key = f"{parent_key}_{key}" if parent_key else key
                if key == 'content' and isinstance(value, str):
                    documents.append(Document(page_content=value, metadata={'source': new_key}))
                else:
                    extract_content(value, new_key)
        elif isinstance(item, list):
            for i, value in enumerate(item):
                new_key = f"{parent_key}_{i}" if parent_key else str(i)
                extract_content(value, new_key)

    extract_content(data)
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size= 1500,
        chunk_overlap= 400,
        length_function=len,
    )
    split_documents = text_splitter.split_documents(documents)
    
    return split_documents

# Function to create ChromaDB client
def create_chroma_db(persist_directory):
    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
    return vectordb

# Function to insert data into ChromaDB
def insert_data_to_chroma(vectordb, documents):
    vectordb.add_documents(documents)
    vectordb.persist()

# Set the file paths
json_file_path = "ai_structured_atomic_habits.json"
persist_directory = "chroma_db3"

# Load and preprocess the data
documents = load_and_preprocess_data(json_file_path)
print(f"Loaded and preprocessed {len(documents)} documents")

# Create ChromaDB instance
vectordb = create_chroma_db(persist_directory)
print(f"Created ChromaDB instance with persist directory: {persist_directory}")

# Insert data into ChromaDB
insert_data_to_chroma(vectordb, documents)
print(f"Data has been successfully loaded into ChromaDB and persisted in {persist_directory}")

In [ ]:
# Function to perform a query and print results
def query_and_print(vectordb, query, k=2):
    print(f"\nQuery: {query}")
    results = vectordb.similarity_search(query, k=k)
    for i, doc in enumerate(results, 1):
        print(f"\nResult {i}:")
        print(f"Source: {doc.metadata['source']}")
        print(f"Content: {doc.page_content}\n")

# Example queries
queries = [
    "What are the four laws of behavior change for creating good habits?",
    "How does environment affect habit formation?",
    "What is the concept of habit stacking?",
    "Explain the Goldilocks Rule in habit formation.",
    "How can one make bad habits more difficult?",
    "What is the role of identity in habit change?",
    "How does the 'Two-Minute Rule' help in overcoming procrastination?",
    "What is the importance of tracking habits?",
    "How can temptation bundling be used to make habits more attractive?",
    "What is the difference between motion and action in habit formation?"
]

# Run queries
for query in queries:
    query_and_print(vectordb, query)

# Bonus: Test a more specific query
specific_query = "How can I apply the four laws of behavior change to develop a habit of daily reading?"
query_and_print(vectordb, specific_query, k=3)

In [ ]:
def expand_query_with_llm(query: str) -> str:
    prompt = """
    You are an AI assistant specializing in habit formation and personal development, with extensive knowledge of James Clear's "Atomic Habits". Your task is to expand a given query to improve search results in a habit-tracking application. The expanded query will be used to retrieve relevant information from a database of personal development and habit formation content.

    Context:
    - The application helps users build better habits based on principles from "Atomic Habits".
    - Key concepts include the four laws of behavior change: make it obvious, attractive, easy, and satisfying.
    - Other important ideas: habit stacking, environment design, identity-based habits, and the two-minute rule.

    Your task:
    1. Analyze the given query for key concepts related to habit formation.
    2. Expand the query by including:
       - Related terms and synonyms
       - Relevant concepts from "Atomic Habits"
       - Specific strategies or techniques mentioned in the book
       - Potential challenges or obstacles related to the habit
       - Measurable outcomes or benefits of the habit

    3. Ensure the expanded query covers multiple aspects of habit formation, including:
       - Habit implementation (cue, craving, response, reward)
       - Habit tracking and measurement
       - Overcoming obstacles and maintaining consistency
       - Long-term behavior change and identity shifts

    4. Format the expanded query as a series of related questions or phrases, separated by semicolons.

    5. Keep the expanded query focused and relevant to the original query's intent.

    Original query: {query}

    Expanded query:
    """

    completion = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": f"Expand the following query:\n\n{query}"}
        ],
        max_tokens=200,
        n=1,
        temperature=0.7,
    )

    expanded_query = completion.choices[0].message.content.strip()
    return expanded_query

test_query = "How can I develop a habit of daily reading?"
expanded = expand_query_with_llm(test_query)
print(f"Original: {test_query}")
print(f"Expanded: {expanded}")


query_and_print(vectordb, expanded, k=2)

In [90]:
def generate_ascend_ai_answer(expanded_query: str, response1: str, response2: str) -> str:
    prompt = f"""
    You are an AI assistant for Ascend AI, a habit-tracking application based on the principles from James Clear's "Atomic Habits". Your role is to provide insightful, practical, and motivating answers to users' questions about habit formation and personal development.

    Given an expanded query and two relevant responses from our knowledge base, your task is to synthesize this information into a comprehensive, coherent answer. Your response should:

    1. Directly address the user's original question and its expanded aspects
    2. Incorporate key insights from "Atomic Habits", including the four laws of behavior change
    3. Provide practical, actionable advice that users can implement immediately
    4. Offer encouragement and motivation to help users stay committed to their habit-building journey
    5. If applicable, suggest how Ascend AI's features (like habit tracking, reminders, or progress visualization) can support the user's goals

    Expanded Query:
    {expanded_query}

    Response 1:
    {response1}

    Response 2:
    {response2}

    Please provide a well-structured, engaging response that combines the most relevant information from these sources and adds your own insights based on the principles of "Atomic Habits".
    """

    try:
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": "Please generate a comprehensive answer based on the given information."}
            ],
            max_tokens=500,
            n=1,
            temperature=0.7,
        )
        return completion.choices[0].message.content.strip()
    except Exception as e:
        return f"An error occurred: {str(e)}"
    
response1 = """Result 1:
Source: laws_0_chapters_1_content
Content: 4. The Diderot Effect:
   - Definition: Tendency for one purchase to lead to additional related purchases
   - Relevance to habits: Demonstrates how behaviors can trigger chain reactions
   - Application: Use principle to create positive habit chains

5. Cue Selection for Habit Stacking:
   Choose cues that:
   a. Occur at same frequency as desired new habit
   b. Happen at a time when you're likely to be successful
   c. Don't conflict with other activities
   d. Consider context and environment where habit will be performed

6. Strategies for Effective Habit Formation:
   a. Use implementation intentions and habit stacking as powerful tools
   b. Create clear, actionable plans with specific methods
   c. Leverage Diderot Effect for comprehensive habit systems
   d. Carefully select cues for successful habit implementation

7. Additional Tips:
   - Be specific about time, location, and behavior
   - Start with small, manageable habits
   - Use visual cues or reminders if needed
   - Regularly review and adjust habit plans

8. Key Takeaways:
   - Specificity crucial in habit formation
   - Leverage existing habits to build new ones
   - Environment and context play vital roles in habit success
   - Regular review and adjustment essential for long-term habit maintenance
"""

response2= """Result 2:
Source: laws_0_chapters_1_content
Content: 4. The Diderot Effect:
   - Definition: Tendency for one purchase to lead to additional related purchases
   - Relevance to habits: Demonstrates how behaviors can trigger chain reactions
   - Application: Use principle to create positive habit chains

5. Cue Selection for Habit Stacking:
   Choose cues that:
   a. Occur at same frequency as desired new habit
   b. Happen at a time when you're likely to be successful
   c. Don't conflict with other activities
   d. Consider context and environment where habit will be performed

6. Strategies for Effective Habit Formation:
   a. Use implementation intentions and habit stacking as powerful tools
   b. Create clear, actionable plans with specific methods
   c. Leverage Diderot Effect for comprehensive habit systems
   d. Carefully select cues for successful habit implementation

7. Additional Tips:
   - Be specific about time, location, and behavior
   - Start with small, manageable habits
   - Use visual cues or reminders if needed
   - Regularly review and adjust habit plans

8. Key Takeaways:
   - Specificity crucial in habit formation
   - Leverage existing habits to build new ones
   - Environment and context play vital roles in habit success
   - Regular review and adjustment essential for long-term habit maintenance

"""
answer = generate_ascend_ai_answer(expanded, response1, response2)


In [ ]:
answer